In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [2]:
# Copyright 2019 Francesco Mannella (francesco.mannella@gmail.com) All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this fileexcept in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

""" Module implementing the EchoStateRNN Cell.
This module provides the EchoStateRNN Cell, implementing the leaky ESN as
described in  http://goo.gl/bqGAJu.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras


class EchoStateRNNCell(keras.layers.Layer):
    """Echo-state RNN cell.
    """

    def __init__(self, units, decay=0.1, alpha=0.5, rho=1.0, sw=1.0, seed=None,
                 epsilon=None, sparseness=0.0,  activation=None, optimize=False,
                 optimize_vars=None, *args, **kwargs):
        """
        Args:
            units (int):  The number of units in the RNN cell.
            decay (float): Decay of the ODE of each unit. Default: 0.1.
            seed (int): seed for random numbers. Default None.
            epsilon (float): Discount from spectral radius 1. Default: 1e-10.
            alpha (float): [0,1], the proporsion of infinitesimal expansion vs infinitesimal rotation
                of the dynamical system defined by the inner weights
            sparseness (float): [0,1], sparseness of the inner weight matrix. Default: 0.
            rho (float): the scale of internal weights
            sw (float): the scale of input weights
            activation (callable): Nonlinearity to use.  Default: `tanh`.
            optimize (bool): whether to optimize variables (see optimize_Vars)
            optimize_vars (list): variables to be optimize ( default None -- all variable are trainable).
        """

        self.seed = seed
        self.units = units
        self.state_size = units
        self.sparseness = sparseness
        self.decay_ = decay
        self.alpha_ = alpha
        self.rho_ = rho
        self.sw_ = sw
        self.epsilon = epsilon
        self._activation = tf.tanh if activation is None else activation
        self.optimize = optimize
        self.optimize_vars = optimize_vars

        super(EchoStateRNNCell, self).__init__(*args, **kwargs)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
          'seed': self.seed,
          'units': self.units,
          'state_size': self.state_size,
          'sparseness': self.sparseness,
          'decay_': self.decay_,
          'alpha_': self.alpha_,
          'rho_': self.rho_,
          'sw_': self.sw_,
          'epsilon': self.epsilon,
          '_activation': self._activation,
          'optimize': self.optimize,
          'optimize_vars': self.optimize_vars,
            })
        return config
    
    def build(self, input_shape):

        # alpha and rho default as tf non trainables
        self.optimize_table = {"alpha": False,
                               "rho": False,
                               "decay": False,
                               "sw": False}

        if self.optimize == True:
            # Set tf trainables
            for var in ["alpha", "rho", "decay", "sw"]:
                if var in self.optimize_vars:
                    self.optimize_table[var] = True
                else:
                    self.optimize_table[var] = False

        # leaky decay
        self.decay = tf.Variable(self.decay_, name="decay",
                                 dtype=tf.float32,
                                 trainable=self.optimize_table["decay"])
        # parameter for dynamic rotation/translation (0.5 means no modifications)
        self.alpha = tf.Variable(self.alpha_, name="alpha",
                                 dtype=tf.float32,
                                 trainable=self.optimize_table["alpha"])

        # the scale factor of the unitary spectral radius
        self.rho = tf.Variable(self.rho_, name="rho",
                               dtype=tf.float32,
                               trainable=self.optimize_table["rho"])

        # the scale factor of the input weights
        self.sw = tf.Variable(self.sw_, name="sw",
                              dtype=tf.float32,
                              trainable=self.optimize_table["sw"])
        
        self.alpha_store = tf.Variable(self.alpha_, name="alpha_store",
                             dtype=tf.float32, trainable=False) 
        
        self.echo_ratio = tf.Variable(1, name="echo_ratio",
                             dtype=tf.float32, trainable=False) 
                
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer=keras.initializers.RandomUniform(-1, 1, seed=self.seed),
            name="kernel", trainable=False)

        self.recurrent_kernel_init = self.add_weight(
            shape=(self.units, self.units),
            initializer=keras.initializers.RandomNormal(seed=self.seed),
            name="recurrent_kernel", trainable=False)
       
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer=tf.zeros_initializer(),
            name="recurrent_kernel_", trainable=False)
    
        self.recurrent_kernel_init.assign(self.setSparseness(self.recurrent_kernel_init))
        self.recurrent_kernel.assign(self.setAlpha(self.recurrent_kernel_init))
        self.echo_ratio.assign(self.echoStateRatio(self.recurrent_kernel))
        self.rho.assign(self.findEchoStateRho(self.recurrent_kernel*self.echo_ratio))
        
        self.built = True

    def setAlpha(self, W):
        W = 0.5*(self.alpha*(W + tf.transpose(W)) + (1 - self.alpha)*(W - tf.transpose(W)))
        return W

    def setSparseness(self, W):
        mask = tf.cast(tf.random.uniform(W.shape, seed=self.seed)
                       < (1 - self.sparseness), dtype=W.dtype)
        W = W * mask
        return W

    def echoStateRatio(self, W):
        eigvals = tf.py_function(np.linalg.eigvals, [W], tf.complex64)
        return tf.reduce_max(tf.abs(eigvals))

    def findEchoStateRho(self, W):
        """Build the inner weight matrix initialixer W  so that
            1 - epsilon < rho(W)  < 1,
            where
            Wd = decay * W + (1 - decay) * I.
            See Proposition 2 in Jaeger et al. (2007) http://goo.gl/bqGAJu.
            See also https://goo.gl/U6ALDd.
            Returns:
                A 2-D tensor representing the
                inner weights of an ESN
        """

        # Correct spectral radius for leaky units. The iteration
        #    has to reach this value
        target = 1.0
        # spectral radius and eigenvalues
        eigvals = tf.py_function(np.linalg.eigvals, [W], tf.complex64)
        x = tf.math.real(eigvals)
        y = tf.math.imag(eigvals)
        # solve quadratic equations
        a = x**2 * self.decay**2 + y**2 * self.decay**2
        b = 2 * x * self.decay - 2 * x * self.decay**2
        c = 1 + self.decay**2 - 2 * self.decay - target**2
        # just get the positive solutions
        sol = (tf.sqrt(b**2 - 4*a*c) - b)/(2*a)
        # and take the minor amongst them
        rho = tf.reduce_min(sol)
        return rho
  
    def clip_variables(self):
        """ clip parameters having been optimized to their limits
        """
        self.decay.assign(tf.clip_by_value(
            self.decay, 0.00000001, 0.25))
        self.alpha.assign(tf.clip_by_value(
            self.alpha, 0.000001, 0.9999999))
        self.rho.assign(tf.clip_by_value(
            self.rho, 0.5, 1.0e100))
        self.sw.assign(tf.clip_by_value(
            self.sw, 0.5, 1.0e100))
    
    def call(self, inputs, states):
        """ Echo-state RNN:
            x = x + h*(f(W*inp + U*g(x)) - x).
        """
        
        rkernel = self.setAlpha(self.recurrent_kernel_init)
        if self.alpha != self.alpha_store:
            self.clip_variables()
            self.echo_ratio.assign(self.echoStateRatio(rkernel))
            self.rho.assign(self.findEchoStateRho(rkernel*self.echo_ratio)) 
            self.alpha_store.assign(self.alpha)

        ratio = self.rho*self.echo_ratio*(1 - self.epsilon)

        prev_output = states[0]
        output = prev_output + self.decay*(
            tf.matmul(
                inputs,
                self.kernel * self.sw) +
            tf.matmul(
                self._activation(prev_output),
                rkernel*ratio)
            - prev_output)

        return self._activation(output), [output]

In [5]:
import pandas as pd
import numpy as np
data_path = '/content/drive/MyDrive/BigData/Project/data.csv'
data = pd.read_csv(data_path)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def get_rain_ro_not(row):
    if row['Precp'] > 0:
        return 1
    else:
        return 0


data['rain'] = data.apply(lambda row: get_rain_ro_not(row), axis=1)

In [ ]:
data

In [7]:
df_inputs = data[['Month', 'ObsTime', 'StnPres', 'SeaPres',
                  'Temperature', 'Td dew point', 'RH', 'WS', 'WD', 'Precp', 'PrecpHour',
                  'SunShine', 'GloblRad', 'Visb', 'Cloud Amount', 'WSGust', 'WDGust']]
df_targets = data[['rain']]

In [8]:
arr_inputs = np.array(df_inputs, dtype=np.float)[0:20000]
arr_targets = np.array(df_targets, dtype=np.float)[0:20000]

val_inputs = np.array(df_inputs, dtype=np.float)[20000:-1]
val_targets = np.array(df_targets, dtype=np.float)[20000:-1]
val_inputs = val_inputs[:,np.newaxis,:]
val_targets = val_targets[:,np.newaxis,:]

segment_length = 1
inputs = []
targets = []
i = 0
for i in range(0, arr_inputs.shape[0] - segment_length):
    tmp = []
    for j in range(0, segment_length):
        tmp.append(arr_inputs[i+j])
    inputs.append(tmp)
    targets.append(arr_targets[i + segment_length])
  # if arr_targets[i + segment_length] == 1:
  #   inputs.append(tmp)
  #   targets.append(arr_targets[i + segment_length])
  #   inputs.append(tmp)
  #   targets.append(arr_targets[i + segment_length])
  #   inputs.append(tmp)
  #   targets.append(arr_targets[i + segment_length])
  #   inputs.append(tmp)
  #   targets.append(arr_targets[i + segment_length])
  
train_inputs = np.array(inputs)
train_targets = np.array(targets)

In [ ]:
print(train_inputs.shape)
print(train_targets.shape)
print(val_inputs.shape)
print(val_targets.shape)

In [ ]:
print(val_targets[val_targets == 0].shape)
print(val_targets[val_targets == 1].shape)
print(train_targets[train_targets == 0].shape)
print(train_targets[train_targets == 1].shape)

In [ ]:
import os

random_seed = np.frombuffer(os.urandom(4), dtype=np.uint32)[0]

cell_0 = EchoStateRNNCell(units=1000,
                        decay=0.1, 
                        epsilon=1e-20,
                        alpha=0.5,
                        seed=random_seed,
                        optimize=True,
                        optimize_vars=["rho", "decay", "alpha", "sw"])

recurrent_layer_0 = keras.layers.RNN(cell_0, input_shape=(segment_length, 17),
                                   return_sequences=True, name="rnn_0")

model = keras.models.Sequential()
model.add(recurrent_layer_0)

for i in range(1 , 20):
    cell = EchoStateRNNCell(units=1000,
                        decay=0.1, 
                        epsilon=1e-20,
                        alpha=0.5,
                        seed=random_seed,
                        optimize=True,
                        optimize_vars=["rho", "decay", "alpha", "sw"])
    recurrent_layer = keras.layers.RNN(cell,
                                   return_sequences=True, name="rnn_" + str(i))
    model.add(recurrent_layer)

cell = EchoStateRNNCell(units=1000,
                      decay=0.1, 
                      epsilon=1e-20,
                      alpha=0.5,
                      seed=random_seed,
                      optimize=True,
                      optimize_vars=["rho", "decay", "alpha", "sw"])

recurrent_layer = keras.layers.RNN(cell, return_sequences=False, name="rnn_last")
model.add(recurrent_layer)

dense_1 = keras.layers.Dense(100, name="dense_1", activation= keras.activations.relu)
dense_2 = keras.layers.Dense(100, name="dense_2", activation= keras.activations.relu)
dense_3 = keras.layers.Dense(20, name="dense_3", activation= keras.activations.relu)
output = keras.layers.Dense(1, name="output", activation= keras.activations.sigmoid)

model.add(dense_1)
model.add(dense_2)
model.add(dense_3)
model.add(output) 
model.summary()

In [10]:
from keras import backend as K
def wbce( y_true, y_pred, weight1=5, weight0=1) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=-1)

model.compile(loss=wbce, optimizer="adam", metrics=['accuracy'])

In [13]:
epochs = 20
hist = model.fit(train_inputs, train_targets, batch_size=64, epochs=epochs, validation_data=(val_inputs, val_targets))
model.save("/content/drive/MyDrive/BigData/Project/model.h5")

Epoch 1/20
313/313 [==============================] - 24s 77ms/step - loss: 0.9426 - accuracy: 0.6719 - val_loss: 0.7580 - val_accuracy: 0.6880
Epoch 2/20
313/313 [==============================] - 24s 76ms/step - loss: 0.9088 - accuracy: 0.6869 - val_loss: 0.7706 - val_accuracy: 0.6608
Epoch 3/20
313/313 [==============================] - 24s 76ms/step - loss: 0.8944 - accuracy: 0.6936 - val_loss: 0.7204 - val_accuracy: 0.7474
Epoch 4/20
313/313 [==============================] - 24s 76ms/step - loss: 0.8678 - accuracy: 0.7062 - val_loss: 0.7525 - val_accuracy: 0.7002
Epoch 5/20
313/313 [==============================] - 24s 75ms/step - loss: 0.8471 - accuracy: 0.7148 - val_loss: 0.7219 - val_accuracy: 0.7349
Epoch 6/20
313/313 [==============================] - 24s 75ms/step - loss: 0.8269 - accuracy: 0.7238 - val_loss: 0.7390 - val_accuracy: 0.7432
Epoch 7/20
313/313 [==============================] - 24s 75ms/step - loss: 0.7986 - accuracy: 0.7376 - val_loss: 0.7761 - val_accuracy:

In [14]:
from keras import backend as K
def wbce( y_true, y_pred, weight1=4, weight0=1) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=-1)

model.compile(loss=wbce, optimizer="adam", metrics=['accuracy'])

In [15]:
import random
epochs = 2
for i in range(0, 10):
    batch_size = 2 ** random.randrange(4, 7) # 16 32 64
    print("Batch size: " + str(batch_size))
    hist = model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=epochs, validation_data=(val_inputs, val_targets))
    model.save("/content/drive/MyDrive/BigData/Project/model.h5")

Batch size: 16
Epoch 1/2
1250/1250 [==============================] - 99s 80ms/step - loss: 0.5733 - accuracy: 0.8046 - val_loss: 0.8019 - val_accuracy: 0.7865
Epoch 2/2
1250/1250 [==============================] - 91s 73ms/step - loss: 0.5960 - accuracy: 0.7992 - val_loss: 0.7316 - val_accuracy: 0.7448
Batch size: 32
Epoch 1/2
625/625 [==============================] - 47s 74ms/step - loss: 0.4695 - accuracy: 0.8412 - val_loss: 0.9107 - val_accuracy: 0.7740
Epoch 2/2
625/625 [==============================] - 46s 73ms/step - loss: 0.4437 - accuracy: 0.8501 - val_loss: 0.9573 - val_accuracy: 0.8035
Batch size: 64
Epoch 1/2
313/313 [==============================] - 24s 76ms/step - loss: 0.3512 - accuracy: 0.8806 - val_loss: 1.1193 - val_accuracy: 0.8147
Epoch 2/2
313/313 [==============================] - 23s 75ms/step - loss: 0.3277 - accuracy: 0.8893 - val_loss: 1.2246 - val_accuracy: 0.8031
Batch size: 64
Epoch 1/2
313/313 [==============================] - 24s 76ms/step - loss: 0.3

In [16]:
correct = 0
wrong = 0
false_postive = 0
false_negative = 0
for i in range(0, val_inputs.shape[0]):
    a = model.predict(val_inputs[i:i+1])[0][0]
    loss = abs(a - val_targets[i][0])
    if (loss < 0.5):
        print("** Correct! ** " + "loss : " + str(loss[0])[0:5] + " target:" + str(val_targets[i][0][0]) + ", predict: " + str(a))
        correct += 1
    else:
        print("** Wrong! ** " + "loss : " + str(loss[0])[0:5]  + ", target:" + str(val_targets[i][0][0]) + ", predict: " + str(a) + ", Acc:" + str(correct / (correct + wrong))[0:4])
        wrong += 1
    if val_targets[i][0][0] == 0:
        false_postive += 1
    else:
        false_negative += 1
  # print("Acc:" + str(correct / (correct + wrong))[0:4])
print("False postive:" + str(false_postive))
print("False negative:" + str(false_negative))

** Correct! ** loss : 0.001 target:0.0, predict: 0.0013855214
** Correct! ** loss : 4.138 target:0.0, predict: 4.1382293e-05
** Correct! ** loss : 8.940 target:0.0, predict: 8.94002e-06
** Correct! ** loss : 0.051 target:0.0, predict: 0.0511649
** Wrong! ** loss : 0.618, target:0.0, predict: 0.618947, Acc:1.0
** Correct! ** loss : 0.000 target:0.0, predict: 0.00074000616
** Wrong! ** loss : 0.670, target:0.0, predict: 0.6707282, Acc:0.83
** Correct! ** loss : 6.593 target:0.0, predict: 6.59391e-06
** Correct! ** loss : 0.009 target:0.0, predict: 0.009157253
** Correct! ** loss : 0.038 target:0.0, predict: 0.038491014
** Correct! ** loss : 8.534 target:0.0, predict: 8.534464e-06
** Correct! ** loss : 0.180 target:0.0, predict: 0.18043661
** Correct! ** loss : 0.000 target:0.0, predict: 0.00019175721
** Wrong! ** loss : 0.657, target:0.0, predict: 0.6575624, Acc:0.84
** Correct! ** loss : 0.002 target:0.0, predict: 0.0026546572
** Correct! ** loss : 0.001 target:0.0, predict: 0.001197589